In [1]:
try:
    print("Elementary imports: ")
    import os
    import json
    import glob
    from PIL import Image, ImageOps
    import argparse
    print("numpy/scipy imports:")
    import numpy as np
    from scipy import signal as sg
    import scipy.ndimage as ndimage
    from scipy.ndimage.filters import maximum_filter
    print("PIL imports:")
    print("matplotlib imports:")
    import matplotlib.pyplot as plt
    from skimage.feature import peak_local_max

except ImportError:
    print("Need to fix the installation")
    raise
print("All imports okay. Yay!")

Elementary imports: 
numpy/scipy imports:
PIL imports:
matplotlib imports:
All imports okay. Yay!


In [2]:
#==============Auxiliary Functions==================

def find_lights(image: np.ndarray, kernel:np.ndarray):

    result = sg.convolve2d(image, kernel)
    max_dots = peak_local_max(result, min_distance=20, num_peaks=10)

    lights_dots = []
    for i in max_dots:
        lights_dots.append([i[1], i[0]])

    return lights_dots

#===================================================

def find_tfl_lights(image_path):

    image = np.array(Image.open(image_path))

    kernel_r = np.array(
        [[-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5],
         [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [1, 1, -0.5], [1, 2, 1],
         [1, 2, 1], [1, 1, 1], [1, 1, 1]])

    kernel_g = np.array(
        [[1, 1, 1], [1, 1, 1], [1, 2, 1], [1, 2, 1], [1, 1, -0.5], [1, 1, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5],
         [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5],
         [-0.5, -0.5, -0.5], [-0.5, -0.5, -0.5], ])


    red_dots = find_lights(image[:, :, 0],kernel_r)
    green_dots = find_lights(image[:, :, 1], kernel_g)

    colors = ['red'] * len(red_dots) + ['green'] * len(green_dots)

    return red_dots + green_dots, colors

In [3]:
def get_cord(image, label):
    
    label = np.array(Image.open(label))
    im = np.array(Image.open(image))
    
    light_dots,colors = find_tfl_lights(image)
    
    TFL_list = []
    not_TFL_list = []    
    
    for dot in light_dots:
        if label[dot[1]][dot[0]] == 19:
            TFL_list.append(dot)
        else:
            not_TFL_list.append(dot)    
                
    leng = min(len(TFL_list),len(not_TFL_list))
    return TFL_list[0:leng] ,not_TFL_list[0:leng]

In [4]:
def add_new_data(filename ,im):
    with open(filename, mode='ab') as fileobj:
        np.array(im, dtype=np.uint8).tofile(fileobj)

In [5]:
def crop_and_save(image, coordinates, is_tfl):
    
    img = Image.open(image)
    img = ImageOps.expand(img, border=41, fill='black')
    for crd in coordinates:
        im = img.crop((round(crd[1]), round(crd[0]), round(crd[1]) + 81, round(crd[0]) + 81))
        if "train" in image:
            add_new_data("data_dir/train/data.bin",im)
            add_new_data("data_dir/train/labels.bin", is_tfl)

        else:
            add_new_data("data_dir/val/data.bin",im)
            add_new_data("data_dir/val/labels.bin", is_tfl)

In [6]:
def build_dataset(image):
    
    label = image.replace("leftImg8bit", "gtFine", 1).replace("_leftImg8bit", "_gtFine_labelIds", 1)
    TFL_list, not_TFL_list = get_cord(image, label)
    crop_and_save(image, TFL_list, 1)
    crop_and_save(image, not_TFL_list, 0)

    
def build_data_from(image_path):
    for root, dirs, files in os.walk(image_path, topdown=True):
        for image in files:
            if image.endswith(".png"):
                build_dataset(os.path.join(root, image))

In [7]:
build_data_from('leftImg8bit/train')


In [8]:
build_data_from('leftImg8bit/val')